In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv(r"emails00.csv")

In [3]:
dataset.columns #Index(['text', 'spam'], dtype='object')

Index(['text', 'spam'], dtype='object')

In [4]:
dataset.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: the stock trading gunslinger fanny i...,1
3,Subject: unbelievable new homes made easy im ...,1
4,Subject: the stock trading gunslinger fanny i...,1


In [5]:
dataset.shape

(5, 2)

In [6]:
#Checking for duplicates and removing them
dataset.drop_duplicates(inplace = True)
#Checking for any null entries in the dataset
print (pd.DataFrame(dataset.isnull().sum()))

      0
text  0
spam  0


In [7]:
dataset.shape  #(5728, 2)

(3, 2)

In [8]:
#Using Natural Language Processing to cleaning the text to make one corpus
# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to C:\Users\Cyber
[nltk_data]     Security\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
dataset['text'][0]

"Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordability : your  ma

In [10]:
#dataset['text'][0]=dataset['text'][0][9:]
dataset['text']=dataset['text'].map(lambda text: text[9:])

In [11]:
dataset['text'][0]

"naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordability : your  marketing b

In [12]:
#Every mail starts with 'Subject :' will remove this from each text 
dataset['text'] = dataset['text'].map(lambda text: text[9:])
dataset['text'] = dataset['text'].map(lambda text:re.sub('[^a-zA-Z0-9]+', ' ',text)).apply(lambda x: (x.lower()).split())

In [13]:
ps = PorterStemmer()
corpus=dataset['text'].apply(lambda text_list:' '.join(list(map(lambda word:ps.stem(word),(list(filter(lambda text:text not in set(stopwords.words('english')),text_list)))))))

In [14]:
type(corpus)

pandas.core.series.Series

In [15]:
type(corpus.values[0])

str

In [16]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus.values).toarray()
y = dataset.iloc[:, 1].values

In [17]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [18]:
dataset.shape

(3, 2)

In [19]:
X_train.shape

(2, 185)

In [20]:
X_test.shape

(1, 185)

In [21]:
y_train.shape

(2,)

In [22]:
y_test.shape

(1,)

In [23]:
# Fitting Naive Bayes classifier to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
classifier.fit(X_train , y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [25]:
y_pred[0]

1

In [26]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [27]:
cm

array([[1]], dtype=int64)

In [28]:
y_test.shape

(1,)

In [29]:
y_pred.shape

(1,)

In [30]:
#saving the model
import pickle
f = open('classifier.pickle', 'wb')
pickle.dump(classifier, f)
f.close()

In [31]:
#loading the model
import pickle
f = open('classifier.pickle', 'rb')
cf = pickle.load(f)
f.close()

In [32]:
#saving the vectorization
import pickle
vec_file = 'vectorizer.pickle'
pickle.dump(cv, open(vec_file, 'wb'))
f.close()

In [33]:
#loading the vectorization
import pickle
e = open('vectorizer.pickle', 'rb')
vc = pickle.load(e)
e.close()